In [1]:
#%run  surrogate_agg.ipynb
%run  surrogate_score.ipynb

In [2]:
def sur_weighted_aggr(preds, sur_score_matrix, ifps_type=None):
    n, m, k = preds.shape
    if ifps_type is None:
        ifps_type = 2 * np.ones(n)
    sur_weighted_pred = -1 * np.ones((n, k))
    #ind_mean_sur_score = np.nanmean(sur_score_matrix, axis=0) 
    scaled_score = np.array(sur_score_matrix) 
    ### for Brier Score
    #scaled_score[scaled_score==-1] = np.NAN
    #scaled_score = 2-scaled_score
    ### for sur score (sur score contains negetive values)
    scaled_score = ((np.nanmax(scaled_score, axis=1)[:, None] - scaled_score) / 
                    (np.nanmax(scaled_score, axis=1)[:, None]-np.nanmin(scaled_score, axis=1)[:, None])) * 2
    scaled_score = scaled_score - (np.nanmedian(scaled_score, axis=1)[:, None]
                                   -np.abs(np.nanmedian(scaled_score, axis=1)[:, None])*0.3)
    ind_mean_sur_score = np.nansum(scaled_score, axis=0)
        
    
    for i in range(n):
        curr_pred = preds[i]
        curr_score = np.array(ind_mean_sur_score)
        curr_score[np.isnan(curr_score) | np.isnan(curr_pred[:, 0])] = -1000
        user_seq = np.argsort(curr_score)
            
        num_answerred_users = np.sum(curr_score>-1000)
        threshold = np.min([np.max([np.min([int(np.ceil(num_answerred_users*0.1)), 20]), 5]),
                            num_answerred_users])
        #print(i, ifps_type[i], num_answerred_users, threshold, np.sum(~np.isnan(ind_mean_sur_score)))
        if threshold<=0: continue
        #print(curr_pred[user_seq[-threshold:]][:, 0])
        #print(curr_pred[user_seq[-num_answerred_users:]][:, 0])
        curr_pred = curr_pred[user_seq[-threshold:]]
        weights = curr_score[user_seq[-threshold:]]
        if (np.max(weights)-np.min(weights)!=0.):
            weights = (weights - np.min(weights))/(np.max(weights)-np.min(weights))
        weights = np.exp(weights)
        sur_weighted_pred[i] = np.average(curr_pred, weights=weights, axis=0)
        #sur_weighted_pred[i] = median_aggr(curr_pred[None, :,:]).squeeze()
        #sur_weighted_pred[i] = entropy_weighted_aggr(curr_pred[None, :,:], ifps_type=[ifps_type[i]])
        #sur_weighted_pred[i] = conf_weighted_aggr(curr_pred[None, :,:], 
        #                                          conf[i, user_seq[-threshold:]][None, :])
        #print(sur_weighted_pred[i])
    #print(np.isnan(scaled_score).any())
    return sur_weighted_pred

In [3]:
import json
import numpy as np
import pandas as pd
import scipy.stats as st
import matplotlib.pyplot as plt

np.set_printoptions(precision=4, suppress=True)

In [4]:
save_file = open('data0928/daily_data_matrix_all_0928.npy','rb')
raw_indicator = np.load(save_file)
raw_preds = np.load(save_file)
raw_conf = np.load(save_file)
raw_wager = np.load(save_file)
raw_se = np.load(save_file)
raw_truth = np.load(save_file)
raw_openday = np.load(save_file)
raw_user_type = np.load(save_file)
raw_user_id = np.load(save_file)
raw_preds_age = np.load(save_file)
save_file.close()

ifps_data = pd.read_csv("data0928/ifps0928_refined.csv")
raw_ifps_type = ifps_data['type'].values


#save_file = open('daily_HFC_data_matrix_agg.npy','rb')
#agg_preds = np.load(save_file)
#agg_se = np.load(save_file)
##agg_openday = np.load(save_file)
### Truth a vector of {1,2} where 1 means the event happen with probability 1
##truth = np.load(save_file)
#save_file.close()

In [16]:
openday = raw_openday[:]
ifps_type = raw_ifps_type[:]
truth = raw_truth[:]


#valid = raw_user_type==1
#valid = raw_user_type==2
valid = raw_user_type==3
#valid = raw_user_type>0
indicator = raw_indicator[:, :, valid]
preds = raw_preds[:, :, valid, :]
conf = raw_conf[:, :, valid]
wager = raw_wager[:, :, valid]
se = raw_se[:, :, valid]
user_type = raw_user_type[valid]
preds_age = raw_preds_age[:, :, valid]


marked_se = np.array(se)
marked_se[marked_se<-0.1] = np.NAN
marked_se[openday<1] = np.NAN

rel_marked_se = marked_se - np.nanmedian(marked_se, axis=2)[:, :, None]

#nan_mark = 1.0 * np.ones_like(raw_openday)
#nan_mark[raw_openday<-0.1] = np.NAN
#se_marked_with_nan *= nan_mark[:, :, None]
#avg_daily_se = np.nanmean(se_marked_with_nan, axis=0)

#print(preds)
print("ifps type:", ifps_type.shape)
print("ground truth:", truth.shape)
print("prediction (day x IFP x user x outcome):", preds.shape)
print("confidence (day x IFP x user): ", conf.shape)
print("wager: ", wager.shape)
print("se of each prediction (day x IFP x user): ", se.shape)
print("status of each IFP (day x IFP): ", openday.shape)
print("days of predictions not being updated (day x IFP x user): ", preds_age.shape)
#print(agg_preds.shape)
#print(openday[35:70].T)
#print(agg_openday[35:70].T)
#print(ifps_data[['map_id','type']])

/Users/heliaguin/anaconda/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:879: RuntimeWarning: All-NaN slice encountered
  result = np.apply_along_axis(_nanmedian1d, axis, a, overwrite_input)


ifps type: (191,)
ground truth: (191,)
prediction (day x IFP x user x outcome): (185, 191, 1175, 5)
confidence (day x IFP x user):  (185, 191, 1175)
wager:  (185, 191, 1175)
se of each prediction (day x IFP x user):  (185, 191, 1175)
status of each IFP (day x IFP):  (185, 191)
days of predictions not being updated (day x IFP x user):  (185, 191, 1175)


In [6]:
from scipy.spatial.distance import cdist, euclidean
def geometric_median(X, eps=1e-5):
    y = np.mean(X, 0)

    while True:
        D = cdist(X, [y])
        nonzeros = (D != 0)[:, 0]

        Dinv = 1 / D[nonzeros]
        Dinvs = np.sum(Dinv)
        W = Dinv / Dinvs
        T = np.sum(W * X[nonzeros], 0)

        num_zeros = len(X) - np.sum(nonzeros)
        if num_zeros == 0:
            y1 = T
        elif num_zeros == len(X):
            return y
        else:
            R = (T - y) * Dinvs
            r = np.linalg.norm(R)
            rinv = 0 if r == 0 else num_zeros/r
            y1 = max(0, 1-rinv)*T + min(1, rinv)*y

        if euclidean(y, y1) < eps:
            return y1

        y = y1

In [10]:
def noisy_label_error_rate(preds_matrix, eval_labels, P0=0.51, kappa=0.02):
#compute the error rates
#multi-testing for selecting the right solution for error rates

    label_matrix = np.zeros_like(preds_matrix, dtype=int)
    label_matrix[preds_matrix>=0.5] = 1
    label_matrix[preds_matrix<0.5] = 2
    label_matrix[preds_matrix<0] = -1
    
    #compute the error rates
    #multi-testing for selecting the right solution for error rates

    num_q = int(len(label_matrix))
    num_w = len(label_matrix[0])
    print(num_q,num_w)

    num_hypo = 11
    cnt_l = 0
    cnt_h = 0
    p0_l = []
    p1_l = []
    P0_est_l = []
    P1_post_l = []
    p0_h = []
    p1_h = []
    P0_est_h = []
    P1_post_h = []

    for t in range(num_hypo):
        kk = int(np.ceil(num_w*2))
        train_data = []
        train_data_Y = []
        for q in range(int(num_q/1)):
            ans_temp = label_matrix[q][:]
            list_taken = [] # extract the list of valid answers. 
            for ii in range(len(ans_temp)):
                if float(ans_temp[ii]) > 0: # -1 is regarded as not taking the task
                    list_taken.append(ii)
            #print list_taken
            
            data_temp = [float(ans_temp[x])-1 for x in list_taken]
            for k in range(kk):
                if len(data_temp)<3: continue
                data_temp2 = random.sample(data_temp,3)
                train_data.append(data_temp2)
        
        p0,p1,P0_est,ind = error_rate_priorfree(P0,train_data)
        print(p0,p1,ind)
        if (max(p0,p1) <= 1) & (min(p0,p1) > 0):
            if ind == 1:
                cnt_l = cnt_l + 1
                p0_l.append(p0)
                p1_l.append(p1)
                P0_est_l.append(P0_est)
                #P1_post_l.append(P1_post)
            else:
                cnt_h = cnt_h + 1
                p0_h.append(p0)
                p1_h.append(p1)
                P0_est_h.append(P0_est)
                #P1_post_h.append(P1_post)

    print(np.mean(p0_l),np.mean(p1_l))
    print(np.mean(p0_h),np.mean(p1_h))

    if cnt_l >= cnt_h:
        p0 = np.mean(p0_l)
        p1 = np.mean(p1_l)
        P0_est = np.mean(P0_est_l)
        #P1_post = np.mean(P1_post_l)
    else:
        p0 = np.mean(p0_h)
        p1 = np.mean(p1_h)
        P0_est = np.mean(P0_est_h)
        #P1_post = np.mean(P1_post_h)

        
    print('Estimated e0,e1,P0:~',p0,p1,P0_est)#,P1_post
    if np.isnan(p0) or np.isnan(p1) or np.isnan(P0_est):
        return np.array([np.NAN, np.NAN])
    return np.array([p0, p1])

def estimate_sur_score(preds, agg_pred, agg_err):
    n, m = preds.shape

    score = np.NAN * np.ones(preds.shape)
    for j in range(m):
        curr_pred = preds[:, j]
        valid = curr_pred>=0.0
        curr_pred = curr_pred[valid]
        answerred_ifps_num = np.sum(valid)
        curr_score = np.zeros(answerred_ifps_num)
        e0, e1 = agg_err
        e1 = e1 * np.ones(answerred_ifps_num)
        e0 = e0 * np.ones(answerred_ifps_num)
        curr_score = (agg_pred[valid] * surrogate_Brier(curr_pred, np.ones(answerred_ifps_num), e0, e1) 
                      + (1-agg_pred[valid]) * surrogate_Brier(curr_pred, np.zeros(answerred_ifps_num), e0, e1))
        score[valid, j] = curr_score
        
    return score

In [7]:
def MBS(preds, truth): # Compute Mean Brier score "for aggregators" over IFPs given predictions and ground truth
    preds = np.array(preds)
    truth = np.array(truth)
    preds[preds<-0.1] = np.NAN
    if np.sum(~np.isnan(preds))<=0:
        return np.NAN, np.ones(len(truth)) * np.NAN
    
    score = np.NAN * np.ones_like(truth)
    resolved = ~np.isnan(truth)
    preds = preds[resolved]
    truth = truth[resolved].astype(int)
    truth = truth - 1
    preds_of_true_label = preds[np.arange(len(truth))[:, None], truth[:, None]].squeeze()
    score[resolved] = np.sum(preds**2, axis=-1) - preds_of_true_label**2 + (1-preds_of_true_label)**2
    return np.nanmean(score), score # return mean BS as well as BS for each IFP

def mean_aggr(preds, weights=None):
    if weights is None: return np.nanmean(preds, axis=1)
    if len(preds.shape)<=2: preds = np.array(preds)[None, :, :]
    num_ifps = preds.shape[0]
    weights = np.array(weights)
    weights = weights.squeeze()
    if len(weights.shape) == 1: weights = np.tile(weights, (num_ifps, 1)) # reproduce weights for each IFP
    weights[np.isnan(preds[:,:,0])] = np.NAN # set weight to np.NAN if there is no prediction. 
    ret = np.nansum(preds * weights[:, :, None], axis=1) / (np.nansum(weights, axis=1)[:, None])
    return ret

def median_aggr(preds):
    n, m, k = preds.shape
    median_pred = -np.ones((n, k))
    for i in range(n):
        users = ~np.isnan(preds[i, :, 0])
        median_pred[i] = geometric_median(preds[i, users, :])
    return median_pred

def top_mean_aggr(preds, weights, top_percent=0.25):
    num_ifps, num_outcomes = preds.shape[0], preds.shape[-1]
    agg_preds = -np.ones((num_ifps, num_outcomes)) # -1 the default missing prediction
    weights = np.array(weights)
    weights = weights.squeeze()
    if len(weights.shape)==1: weights = np.tile(weights, (num_ifps, 1)) # reproduce weights for each IFP
    for i in range(num_ifps):
        answered_users = preds[i, :, 0]>-0.1
        curr_preds = preds[i, answered_users, :]
        curr_weights = weights[i, answered_users]
        num_users = len(curr_weights)
        #threshold = int(np.round(np.min(0.1 * num_users, 20)))
        threshold = int(np.round(top_percent * num_users))
        user_rank = np.argsort(curr_weights)
        selected_users = user_rank[-threshold:]
        agg_preds[i] = np.average(curr_preds[selected_users], weights=curr_weights[selected_users], axis=0)
        #agg_preds[i] = geometric_median(curr_preds[selected_users])
    
    return agg_preds

In [32]:
def get_error_rate(preds, agg_preds, trials=100):
    n, m = preds.shape
    agg_labels_multi_trials = np.zeros((n, trials))
    for i in range(trials):
        labels = 2 - (np.random.rand(n) < agg_preds)
        agg_labels_multi_trials[:, i] = labels
    # set the first trial, the label generated by majority voting of the agg_pred
    agg_labels_multi_trials[:, 0] = 2 - (0.5 < agg_preds)
    #error_rate = noisy_label_error_rate(preds, eval_labels=agg_labels_multi_trials, P0=P0)
    err_agg_majority_0 = noisy_label_error_rate(preds, eval_labels=agg_labels_multi_trials, P0=0.51)
    err_agg_majority_1 = noisy_label_error_rate(preds, eval_labels=agg_labels_multi_trials, P0=0.49)
    err_agg_majority_0[np.isnan(err_agg_majority_0)] = 1000
    err_agg_majority_1[np.isnan(err_agg_majority_1)] = 1000
    if np.sum(err_agg_majority_0) < np.sum(err_agg_majority_1):
        return err_agg_majority_0
    return err_agg_majority_1

# Aggregation
t, n, m, k = preds.shape
print(t, n, m, k)

# parameters
r = 0.25 # coefficient of weight updates in Multiplicative Weights


num = np.zeros((t,n))


BS_mean = np.NAN * np.ones((t, n))
MBS_mean = np.NAN * np.ones(t)
BS_median = np.NAN * np.ones((t, n))
MBS_median = np.NAN * np.ones(t)

BS_mw_bench_top_mean = np.NAN * np.ones((t, n))
MBS_mw_bench_top_mean = np.NAN * np.ones(t)

BS_mw_filled_top_mean = np.NAN * np.ones((t, n))
MBS_mw_filled_top_mean = np.NAN * np.ones(t)

BS_sum_RSSR_top_mean = np.NAN * np.ones((t, n))
MBS_sum_RSSR_top_mean = np.NAN * np.ones(t)

SE = np.NAN * np.ones((t, n))


mw_bench = np.ones(m)
mw_filled = np.ones(m)

err_agg = np.array([1000, 1000])
for h in range(t):
#    if h<100: continue
    if h%20==0: print(h)
    curr_pred = np.array(preds[h])
    num[h] = np.sum(curr_pred[:, :, 0]>-0.1, axis=1)*(openday[h]>=1) #compute # of preds of open IFPs on day h
    curr_ifps = num[h]>0 # find out IFPs with postive # of preds
    if np.sum(curr_ifps)<=0: continue
        
    ### Aggregation Part
    preds_for_aggr = curr_pred[curr_ifps]
    preds_for_aggr[preds_for_aggr<-0.1] = np.NAN
    
    mean_pred = -np.ones((n, k))
    median_pred = -np.ones((n, k))
    mw_bench_top_mean_pred = -np.ones((n, k))
    mw_filled_top_mean_pred = -np.ones((n, k))
    sum_RSSR_top_mean_pred = -np.ones((n, k))
    
    mean_pred[curr_ifps] = mean_aggr(preds_for_aggr)
    #median_pred[curr_ifps] = median_aggr(preds_for_aggr)
    mw_bench_top_mean_pred[curr_ifps] = top_mean_aggr(preds_for_aggr, weights=mw_bench, top_percent=1)
    mw_filled_top_mean_pred[curr_ifps] = top_mean_aggr(preds_for_aggr, weights=mw_filled, top_percent=0.15)

    MBS_mean[h], BS_mean[h] = MBS(mean_pred, truth)
    #MBS_median[h], BS_median[h] = MBS(median_pred, truth)
    MBS_mw_bench_top_mean[h], BS_mw_bench_top_mean[h] = MBS(mw_bench_top_mean_pred, truth)
    MBS_mw_filled_top_mean[h], BS_mw_filled_top_mean[h] = MBS(mw_filled_top_mean_pred, truth)
    
    
    ### Multiplicative Weight Updates Part
    for i in range(n):
        if not (openday[h, i]==-2 and openday[h-1, i]>-2): continue
        if openday[h-1, i]<1: continue # there is one IFP resolved before being open
        # Use the IFP i which is just resolved to update weights

        # Compute mw_bench
        #answered_users = preds[h, i, :, 0]>-0.1
        answered_users = ~np.isnan(np.nanmean(marked_se[:, i, :], axis=0))
        se_of_users = np.nanmean(marked_se[:, i, answered_users], axis=0)
        for ii, xx in enumerate(se_of_users):
            if np.isnan(xx):
                print(openday[:, i])
                print(se[:, i, answered_users][:, ii])
                print(marked_se[:, i, answered_users][:, ii])
        if np.isnan(se_of_users).any(): 
            asdasd
        min_se, max_se = np.min(se_of_users), np.max(se_of_users)
        if min_se == max_se: continue
        normalized_se = (se_of_users-min_se)/(max_se-min_se) # normalize se into [0,1], still smaller <=> better
        mw_bench[answered_users] *= (1-r*normalized_se)
        #print(mw_bench[answered_users])
        
        # Compute mw_filled
        se_of_users = np.zeros(m)
        se_of_users[answered_users] = np.nanmean(marked_se[:, i, answered_users], axis=0)
        num_outcomes = ifps_type[i]
        se_of_users[~answered_users] = (1 - 1/num_outcomes)**2+(num_outcomes-1)*(1/num_outcomes)**2
        min_se, max_se = np.min(se_of_users), np.max(se_of_users)
        if min_se == max_se: continue
        normalized_se = (se_of_users-min_se)/(max_se-min_se) # normalize se into [0,1], still smaller <=> better
        mw_filled *= (1-r*normalized_se)
        
    ### Surrogate Error Rate Computation Part
    # Using all predictions including those closed to calculate error rate
    # Select binary IFPs with predictions made by at least 20 users. 
    ifps_for_SSR = (ifps_type == 2) & (np.sum(preds[h, :, :, 0]>-0.1, axis=1)>=20)
    num_ifps_for_SSR = len(ifps_for_SSR)
    if num_ifps_for_SSR>0:
        # One-dimension aggregation of prediction
        agg_preds_for_SSR = mean_aggr(preds[h, ifps_for_SSR])[:, 0] # only the prob of outcome 0 is recorded
        preds_for_SSR = preds[h, ifps_for_SSR, :, 0] # Use prob on Outcome 0 to represnent a pred
                
    if h%10==0 and h>=60 and num_ifps_for_SSR>0:
        print("Compute surrogate error rate on Day ", h)
        err_agg = get_error_rate(preds_for_SSR, agg_preds_for_SSR, trials=100)
        print(err_agg)
        
    if num_ifps_for_SSR>0:    
        sur_score_matrix = estimate_sur_score(preds_for_SSR, agg_preds_for_SSR, err_agg)
        print(sur_score_matrix.shape)
        
    if (np.sum(err_agg)<0.55):
        print()
        sum_RSSR_top_mean_pred[curr_ifps] = sur_weighted_aggr(preds_for_aggr, sur_score_matrix,
                                                              ifps_type[curr_ifps])
        MBS_sum_RSSR_top_mean[h], BS_sum_RSSR_top_mean[h] = MBS(sum_RSSR_top_mean_pred, truth)
        print('Day '+str(h)+" mw_filled_top_mean MBS:", MBS_mw_filled_top_mean[h])
        print('Day '+str(h)+" sum_RSSR_top_mean MBS:", MBS_sum_RSSR_top_mean[h])
    else: sum_RSSR_top_mean_pred[curr_ifps] = mw_filled_top_mean_pred[curr_ifps]
    MBS_sum_RSSR_top_mean[h], BS_sum_RSSR_top_mean[h] = MBS(sum_RSSR_top_mean_pred, truth)
    
print('Completed!')

185 191 1175 5
0
(0, 1175)
(6, 1175)
(8, 1175)
(8, 1175)
(9, 1175)


/Users/heliaguin/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:87: RuntimeWarning: Mean of empty slice


(9, 1175)
(9, 1175)
(9, 1175)
(12, 1175)
(13, 1175)
(14, 1175)
(14, 1175)
(14, 1175)
(14, 1175)
(14, 1175)
(14, 1175)
(15, 1175)
(16, 1175)
(16, 1175)
(16, 1175)
20
(17, 1175)
(17, 1175)
(17, 1175)
(17, 1175)
(17, 1175)
(17, 1175)
(18, 1175)
(18, 1175)
(19, 1175)
(23, 1175)
(25, 1175)
(26, 1175)
(27, 1175)
(27, 1175)
(27, 1175)
(29, 1175)
(31, 1175)
(31, 1175)
(32, 1175)
(33, 1175)
40
(34, 1175)
(34, 1175)
(34, 1175)
(36, 1175)
(36, 1175)
(36, 1175)
(38, 1175)
(38, 1175)
(38, 1175)
(38, 1175)
(39, 1175)
(39, 1175)
(39, 1175)
(39, 1175)
(40, 1175)
(40, 1175)
(40, 1175)
(46, 1175)
(48, 1175)
(48, 1175)
60
Compute surrogate error rate on Day  60
48 1175
0.848157446314 0.797873012091 -1
0.851494983417 0.793339507656 -1
0.843822584835 0.811655045297 -1
0.845335315499 0.797146965048 -1
0.844531928611 0.812009302037 -1
0.850636677042 0.788279087449 -1
0.849308122772 0.80435786733 -1
0.84414125324 0.817414477714 -1
0.849151817766 0.79562011932 -1
0.846519188002 0.806352972551 -1
0.845617161861

/Users/heliaguin/anaconda/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/heliaguin/anaconda/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


0.199145961062 0.151217209027 1
0.20539601784 0.149523527541 1
0.19328711114 0.154819008039 1
0.212767943907 0.149682682203 1
0.181432367939 0.153809169998 1
0.180206577722 0.1555360028 1
0.202883898871 0.151168300805 1
0.229185772725 0.144108462722 1
0.220828833417 0.145872824465 1
0.199853691314 0.151832693823 1
0.19915759041 0.152751472625 1
0.202195069668 0.150938304913
nan nan
Estimated e0,e1,P0:~ 0.202195069668 0.150938304913 0.282988753429
[ 0.2022  0.1509]
(48, 1175)

Day 60 mw_filled_top_mean MBS: 0.374420659641
Day 60 sum_RSSR_top_mean MBS: 0.377785368941
(48, 1175)

Day 61 mw_filled_top_mean MBS: 0.37523635921
Day 61 sum_RSSR_top_mean MBS: 0.373122697653
(48, 1175)

Day 62 mw_filled_top_mean MBS: 0.362343424567
Day 62 sum_RSSR_top_mean MBS: 0.361139247648
(49, 1175)

Day 63 mw_filled_top_mean MBS: 0.398616567324
Day 63 sum_RSSR_top_mean MBS: 0.400236772496
(50, 1175)

Day 64 mw_filled_top_mean MBS: 0.362636424131
Day 64 sum_RSSR_top_mean MBS: 0.374306968196
(50, 1175)

Day 6

0.197905232631 0.143219806965 1
0.201782414436 0.14213746827 1
0.20084861255 0.141751219105 1
0.196904389707 0.143346672266 1
0.20738467465 0.139935171607 1
0.203480938332 0.140500147971 1
0.196729777807 0.143805887824 1
0.199519708129 0.142092299969
nan nan
Estimated e0,e1,P0:~ 0.199519708129 0.142092299969 0.307869196965
[ 0.1995  0.1421]
(70, 1175)

Day 100 mw_filled_top_mean MBS: 0.336300033428
Day 100 sum_RSSR_top_mean MBS: 0.320910523831
(70, 1175)

Day 101 mw_filled_top_mean MBS: 0.33777870477
Day 101 sum_RSSR_top_mean MBS: 0.321726067582
(70, 1175)

Day 102 mw_filled_top_mean MBS: 0.338094536467
Day 102 sum_RSSR_top_mean MBS: 0.321380050871
(70, 1175)

Day 103 mw_filled_top_mean MBS: 0.336793520749
Day 103 sum_RSSR_top_mean MBS: 0.316400743165
(70, 1175)

Day 104 mw_filled_top_mean MBS: 0.332794338695
Day 104 sum_RSSR_top_mean MBS: 0.310089106034
(74, 1175)

Day 105 mw_filled_top_mean MBS: 0.356237182689
Day 105 sum_RSSR_top_mean MBS: 0.339217558833
(74, 1175)

Day 106 mw_fille

0.178681593402 0.129561271794 1
0.176034552245 0.130327080751 1
0.186006763309 0.128268127398 1
0.170568675889 0.133603604436 1
0.169454380447 0.132870200962 1
0.169866983135 0.133709350182 1
0.175102800928 0.130852193773
nan nan
Estimated e0,e1,P0:~ 0.175102800928 0.130852193773 0.307598578749
[ 0.1751  0.1309]
(81, 1175)

Day 140 mw_filled_top_mean MBS: 0.300824000998
Day 140 sum_RSSR_top_mean MBS: 0.249143545499
(81, 1175)

Day 141 mw_filled_top_mean MBS: 0.297533857426
Day 141 sum_RSSR_top_mean MBS: 0.248697461747
(81, 1175)

Day 142 mw_filled_top_mean MBS: 0.294123107904
Day 142 sum_RSSR_top_mean MBS: 0.244417822844
(81, 1175)

Day 143 mw_filled_top_mean MBS: 0.293144217872
Day 143 sum_RSSR_top_mean MBS: 0.243578814135
(81, 1175)

Day 144 mw_filled_top_mean MBS: 0.291334732142
Day 144 sum_RSSR_top_mean MBS: 0.241797439022
(81, 1175)

Day 145 mw_filled_top_mean MBS: 0.290594587692
Day 145 sum_RSSR_top_mean MBS: 0.234541989332
(81, 1175)

Day 146 mw_filled_top_mean MBS: 0.2965662101

0.178131614455 0.120349359198 1
0.19964084451 0.115355488954 1
0.182996443993 0.118186387227 1
0.190374226832 0.117465783003 1
0.182234247783 0.11872239131 1
0.188454385658 0.117649730865
nan nan
Estimated e0,e1,P0:~ 0.188454385658 0.117649730865 0.30012961577
[ 0.1885  0.1176]
(88, 1175)

Day 180 mw_filled_top_mean MBS: 0.163468359181
Day 180 sum_RSSR_top_mean MBS: 0.113076506686
(88, 1175)

Day 181 mw_filled_top_mean MBS: 0.154701601535
Day 181 sum_RSSR_top_mean MBS: 0.109542779995
(88, 1175)

Day 182 mw_filled_top_mean MBS: 0.114875774353
Day 182 sum_RSSR_top_mean MBS: 0.0561281105572
(88, 1175)

Day 183 mw_filled_top_mean MBS: 0.0767537796658
Day 183 sum_RSSR_top_mean MBS: 0.0315009771813
(88, 1175)

Day 184 mw_filled_top_mean MBS: 0.0630684526322
Day 184 sum_RSSR_top_mean MBS: 0.024013512068
Completed!


In [15]:
start_day = 60
print(np.array([np.nanmean([np.nanmean(BS_mean[start_day:, i]) for i in range(n)]),
                np.nanmean([np.nanmean(BS_median[start_day:, i]) for i in range(n)]),
                np.nanmean([np.nanmean(BS_mw_bench_top_mean[start_day:, i]) for i in range(n)]),
                np.nanmean([np.nanmean(BS_mw_filled_top_mean[start_day:, i]) for i in range(n)]),
                np.nanmean([np.nanmean(BS_sum_RSSR_top_mean[start_day:, i]) for i in range(n)])
               ]))

baseline = np.array([np.nanmean(BS_mw_filled_top_mean[start_day:, i]) for i in range(n)])
sur = np.array([np.nanmean(BS_sum_RSSR_top_mean[start_day:, i]) for i in range(n)])
valid = (~np.isnan(baseline)) & (~np.isnan(sur))
baseline = baseline[valid]
sur = sur[valid]
t_score, p_value = st.ttest_1samp(sur - baseline, popmean=0)
p_value = p_value / 2
print("t, p value(one_tail): ", t_score, p_value)

t_score, p_value = st.ttest_1samp(MBS_sum_RSSR_top_mean - MBS_mw_filled_top_mean, popmean=0)
p_value = p_value / 2
print("t, p value(one_tail): ", t_score, p_value)

[ 0.4022     nan  0.4099  0.3465  0.3419]
t, p value(one_tail):  -0.62915555147 0.265069459329
t, p value(one_tail):  -9.92404190157 3.66352111436e-19


/Users/heliaguin/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: Mean of empty slice
  
/Users/heliaguin/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: Mean of empty slice
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/heliaguin/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: Mean of empty slice
  after removing the cwd from sys.path.
/Users/heliaguin/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: Mean of empty slice
  """
/Users/heliaguin/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:6: RuntimeWarning: Mean of empty slice
  
/Users/heliaguin/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: Mean of empty slice
  if __name__ == '__main__':
/Users/heliaguin/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: Mean of empty slice
  # Remove the CWD from sys.pat